In [1]:
import torch.nn as nn
import torch
from torchvision import transforms, datasets, utils
import os
import sys
import json
import time
import torch
import torch.nn as nn
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
from torchsummaryX import summary
from model import AlexNet

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=False):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2), 
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                 
            nn.Conv2d(48, 128, kernel_size=5, padding=2),          
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                
            nn.Conv2d(128, 192, kernel_size=3, padding=1),        
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),       
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),       
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),               
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


In [3]:
def train():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
        "val": transforms.Compose([transforms.Resize((224, 224)),  
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}
    data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
    image_path = os.path.join(data_root, "data_set", "flower_data")  # flower data set path
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
    train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train"),
                                         transform=data_transform["train"])
    train_num = len(train_dataset)

    # {'daisy':0, 'dandelion':1, 'roses':2, 'sunflower':3, 'tulips':4}
    flower_list = train_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in flower_list.items())
    # write dict into json file
    json_str = json.dumps(cla_dict, indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 32
    nw =0
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(root=os.path.join(image_path, "val"),
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=4, shuffle=False,
                                                  num_workers=nw)

    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))


    net = AlexNet(num_classes=2, init_weights=True)
    xx=torch.zeros(1,3,224,224)
    summary(net,xx)
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.0002)

    epochs = 30
    save_path = './AlexNet.pth'
    best_acc = 0.0
    train_steps = len(train_loader)
    timlist=[]
    t_loss=[]
    val_a=[]
    nowtim=time.perf_counter()
    timlist.append(nowtim)
    val_a.append(0)
    t_loss.append(1)
    for epoch in range(epochs):
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / val_num
        tim=time.perf_counter()
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f time:%.3lf' %
              (epoch + 1, running_loss / train_steps, val_accurate,tim-nowtim))
        nowtim=tim
        timlist.append(nowtim)
        val_a.append(val_accurate)
        t_loss.append(running_loss / train_steps)
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
    print('Finished Training')
    return timlist,val_a,t_loss


In [4]:
def test():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_transform = transforms.Compose(
        [transforms.Resize((224, 224)),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # load image
    #img_path = "../tulip.jpg"
    #assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
    #img = Image.open(img_path)
    
    
    path_img = './testimages/'
    ls = os.listdir(path_img)
    pre=[]
    for i in ls:
        img = Image.open(f"./testimages/{i}")

        #plt.imshow(img)
        # [N, C, H, W]
        img = data_transform(img)
        # expand batch dimension
        img = torch.unsqueeze(img, dim=0)

        # read class_indict
        json_path = './class_indices.json'
        assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

        with open(json_path, "r") as f:
            class_indict = json.load(f)

        # create model
        model = AlexNet(num_classes=2).to(device)

        # load model weights
        weights_path = "./AlexNet.pth"
        assert os.path.exists(weights_path), "file: '{}' dose not exist.".format(weights_path)
        model.load_state_dict(torch.load(weights_path))

        model.eval()
        with torch.no_grad():
            # predict class
            output = torch.squeeze(model(img.to(device))).cpu()
            predict = torch.softmax(output, dim=0)
            predict_cla = torch.argmax(predict).numpy()

        print_res = "class: {}   prob: {:.3}".format(class_indict[str(predict_cla)],
                                                     predict[predict_cla].numpy())
        if class_indict[str(predict_cla)]=='male':
            pre.append(1)
        else:
            pre.append(-1)
    return pre

In [ ]:
a,b,c=train()

using cpu device.
Using 0 dataloader workers every process
using 2701 images for training, 299 images for validation.
                              Kernel Shape      Output Shape     Params  \
Layer                                                                     
0_features.Conv2d_0        [3, 48, 11, 11]   [1, 48, 55, 55]    17.472k   
1_features.ReLU_1                        -   [1, 48, 55, 55]          -   
2_features.MaxPool2d_2                   -   [1, 48, 27, 27]          -   
3_features.Conv2d_3        [48, 128, 5, 5]  [1, 128, 27, 27]   153.728k   
4_features.ReLU_4                        -  [1, 128, 27, 27]          -   
5_features.MaxPool2d_5                   -  [1, 128, 13, 13]          -   
6_features.Conv2d_6       [128, 192, 3, 3]  [1, 192, 13, 13]   221.376k   
7_features.ReLU_7                        -  [1, 192, 13, 13]          -   
8_features.Conv2d_8       [192, 192, 3, 3]  [1, 192, 13, 13]   331.968k   
9_features.ReLU_9                        -  [1, 192, 13, 

In [82]:
pre=test()

In [99]:
a.extend(b)
a.extend(c)
a=np.array(a)
a=np.resize(a,(3,3))
print(a)
df = pd.DataFrame(a)
df.to_csv('Alexnetdata.csv',index= False, header= False)

[[1 1 1]
 [2 2 2]
 [3 3 3]]


In [83]:
path_img = './testimages/'
ls = os.listdir(path_img)
name=[]
for i in ls:
    name.append(i)

In [84]:
import pandas as pd
out_dict = {
    'image_id':list(name),
    'is_male':list(pre)
}
out = pd.DataFrame(out_dict)
out.to_csv('Alexnet.csv',index=False)